In [1]:
!pip install -U transformers trl datasets torch==2.6.0 accelerate bitsandbytes peft langchain langchain_huggingface pymongo openai pyngrok fastapi uvicorn

In [2]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
import os
import nest_asyncio
from google.colab import userdata
from Rag_Chatbot import RAGChatbot

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI(title="HCMIU RAG Chatbot API")

# Initialize RAG chatbot
chatbot = RAGChatbot(
    mongodb_uri=userdata.get('mongodb'),
    openai_api_key=userdata.get('OpenAI')
)

class Query(BaseModel):
    text: str

@app.get("/")
async def read_root():
    return {"message": "HCMIU RAG Chatbot API is running!"}

@app.post("/chat")
async def chat(query: Query):
    try:
        response = chatbot.chat(query.text)
        return {"response": response}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

def start_server():
    # Set up ngrok authtoken
    NGROK_AUTH_TOKEN = userdata.get('ngrok_auth_token')
    if NGROK_AUTH_TOKEN:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        print("ngrok authtoken set.")
    else:
        print("NGROK_AUTH_TOKEN not found in Colab secrets. Please add it to use ngrok.")

    try:
        # Kill any existing ngrok tunnels
        ngrok.kill()
        print("Existing ngrok tunnels killed.")

        # Start a new tunnel
        public_url = ngrok.connect(addr="8000", proto="http")
        print(f"FastAPI Public URL: {public_url}")

        # Start FastAPI server
        uvicorn.run(app, host="0.0.0.0", port=8000, loop="asyncio")
    except Exception as e:
        print(f"Error starting server: {e}")

if __name__ == "__main__":
    start_server()

Loading model and tokenizer...


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Model and tokenizer loaded successfully!
ngrok authtoken set.
Existing ngrok tunnels killed.
FastAPI Public URL: NgrokTunnel: "https://f8a4-34-124-214-1.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [2330]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)



Searching for 'Xin chào, tôi có anh ruột là sinh viên ở trường, vậy tôi được giảm bao nhiêu tiền học phí nếu vào học ở trường ?' in document type: policy
Found 0 relevant documents
INFO:     2402:800:6370:9907:29b8:d128:d659:6a19:0 - "POST /chat HTTP/1.1" 200 OK



Searching for 'và ba tôi là một thương binh liệt sĩ, vậy tôi được sẽ được giảm thêm bao nhiêu ?' in document type: policy
Found 0 relevant documents
INFO:     2402:800:6370:9907:29b8:d128:d659:6a19:0 - "POST /chat HTTP/1.1" 200 OK

Searching for 'Tôi muốn tham khảo chương trình học của ngành Logistics năm nhất' in document type: course_structure
Found 10 relevant documents

📄 Result 1
Title: Ngành Logistics và Quản lý chuỗi cung ứng: Năm 4
Document Type: course_structure
Combined Score: 8.0675
Vector Score: 0.0000
Text Score: 8.0675
Content Preview: Học kì 1:
IS111IU-Capstone 1-3
IS033IU-Multi-Criteria Decision Making-3
IS067IU-International Transportation & Logistics-3
IS107IU-Supply Chain Modelling and Simulation-3
IS062IU-E-Logistics in Supply...
--------------------------------------------------

📄 Result 2
Title: Ngành Logistics và Quản lý chuỗi cung ứng: Năm 2
Document Type: course_structure
Combined Score: 7.5658
Vector Score: 0.0000
Text Score: 7.5658
Content Preview: Học kì 1

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2330]
